This resulted dataframe below returns the CV and p values of the significant genes overlapped between the GTEx adipose subcutaneous dataset and the four day young and old (regardless of genders). The same program can also be applied to other set of comparsions.

In [1]:
import pandas as pd
import numpy as np

# functions used in the function's file
# from scipy.stats import levene
# from statsmodels.stats.multitest import multipletests

# import the file that stores functions
import Analysis_Functions 


# Load datasets then change the following to be those processed dataframes, 
# need include age in adipose_subcutaneous data, and set "Description" to be adipose_data's index
adipose_data = adipose_subcutaneous # GTEx data
gene_metadata = gene_info
counts_8day_data = counts_eight_day # for eight day data's CV and p value calculation
counts_4day_data = counts_four_day

# define subsets for four-day dataset analysis
male_young_4day = counts_4day_data.filter(regex='MY1', axis=1).iloc[:, :-1]
male_old_4day = counts_4day_data.filter(regex='OM1', axis=1)
female_young_4day = counts_4day_data.filter(regex='^(?!.*trl).*((YF)|(FY))', axis=1)
female_old_4day = counts_4day_data.filter(regex='^(?!.*trl).*((OF)|(FO))', axis=1)
combined_young_4day = pd.concat([male_young_4day, female_young_4day], axis=1)
combined_old_4day = pd.concat([male_old_4day, female_old_4day], axis=1)

# perform Levene's test and calculate CV ratios for four-day dataset
levene_results_4day = Analysis_Functions.levene_test_between_groups(combined_young_4day, combined_old_4day)
levene_results_4day['cv_ratio_combined'] = Analysis_Functions.calculate_cv_ratio(combined_young_4day, combined_old_4day)

# find significant genes with such thresholds (CV fold change>1 and p value < 0.05) which is the same threshold for both datasets
significant_proteins = levene_results_4day[(levene_results_4day['cv_ratio_combined'] > 1) & (levene_results_4day['p_value'] < 0.05)]

# filter the ones that are significant in fourday also in the adipose GTEx data, only perform tests on those to reduce computational cost
# also can remove this step if willing to check all significant genes in GTEx data alone
filtered_adipose_data = adipose_data.loc[significant_proteins.set_index('protein').index.intersection(adipose_data.index)]
filtered_adipose_data.loc['age'] = adipose_data.loc['age']
# remove duplicates if necessary
#filtered_adipose_data = filtered_adipose_data[~filtered_adipose_data.index.duplicated(keep='first')] # or don't leave any

# define age-specific subsets for adipose data, 20-29 to be young, and 60-69 & 70-79 to be old, consistent with other age set used in the article
adipose_age_20_29 = filtered_adipose_data.loc[:, filtered_adipose_data.loc['age'] == '20-29']
adipose_age_60_79 = filtered_adipose_data.loc[:, (filtered_adipose_data.loc['age'] == '60-69') | (filtered_adipose_data.loc['age'] == '70-79')]

# perform Levene's test and calculate CV ratios for GTEx adipose data age groups
adipose_age_20_29, adipose_age_60_79 = adipose_age_20_29.iloc[:-1, :], adipose_age_60_79.iloc[:-1, :]
common_proteins = adipose_age_20_29.dropna().index.intersection(adipose_age_60_79.dropna().index)
adipose_age_20_29 = adipose_age_20_29.loc[common_proteins].astype(float)
adipose_age_60_79 = adipose_age_60_79.loc[common_proteins].astype(float)

adipose_results = Analysis_Functions.levene_test_between_groups(adipose_age_20_29, adipose_age_60_79)
adipose_results['cv_ratio'] = Analysis_Functions.calculate_cv_ratio(adipose_age_20_29, adipose_age_60_79)

# find final intersections
final_intersection = adipose_results[(adipose_results['p_value'] < 0.05) & (adipose_results['cv_ratio'] > 1)].set_index('protein')


C:\Users\17599\AppData\Local\Temp\ipykernel_50960\2271153200.py:118: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,

In [2]:
final_intersection

,p_value,cv_ratio
protein,,
CNBP,1.848700e-02,1.261849
MYL12B,4.857591e-02,1.280582
IRS2,4.499540e-02,1.436724
RHEB,3.997347e-02,1.543844
CNTFR,5.167535e-05,1.206586
TFPI,6.801404e-03,1.223747
DHRS3,4.632481e-02,1.207099
HCAR2,2.854773e-02,1.700345
TWSG1,5.042802e-03,1.036534
